# Final Project: Challenge 4
Contributions From: Ryan Cottone

**Difficulty**: <span style="color:orange">$\bullet \bullet$</span>

Welcome to the 4th challenge! Here, you will be taking the role of an **active attacker** rather than a passive one. You will be given the function **interceptMessage(data)** within the class **Interceptor** which intercepts every communication between the client and server will be called with.

For example, if the client sends "data", then the function interceptMessage("data") gets called and the server gets the **output** of interceptMessage("data") as the data.

On top of that, the following functions have been changed:
- **generateSignedECDH**
- **verifyECDHMessage**

Recover the user's password.

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
%run client.ipynb
%run server.ipynb
from utils import *

In [ ]:
ellipticCurve = [0, 7]
ellipticCurveModulus = 115792089237316195423570985008687907853269984665640564039457584007908834671663 
generatorPoint = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)
generatorPointOrder =115792089237316195423570985008687907852837564279074904382605163141518161494337

Here's a copy of the new `generateSignedECDH` and `verifyECDHMessage` functions:

In [ ]:
def generateSignedECDHMessage(self):
    nP, n = generateECDH(self.GENERATOR_POINT, self.ELLIPTIC_CURVE, self.ELLIPTIC_CURVE_MODULUS, self.GENERATOR_POINT_ORDER)
        
    self.ECDH_SECRET = n
        
    return nP, 0
    
def verifyECDHMessage(self, P, signature, publicKey):
    """
    Verifies point P was signed by the given publicKey (e, N).
    """
        
    return True # Always verify the signature as correct

## Tips

Make sure to use the given `self.fakeServerSecret` if you need to generate a ECDH secret for some sort of attack, as the secret message is encoded such that the RNG will generate the required secret.

### Fill this in:

In [ ]:
class Interceptor:
    def __init__(self):
        self.clientPassword = None
        self.fakeServerSecret = random.randint(1, generatorPointOrder - 1) # DON'T CHANGE THIS
        
    def interceptMessage(self, message):
        """
        The message will be of the form 
        { 
            sender: 'client/server',
            receiver: 'client/server',
            type: 'certificate/ECDH_Parameters/ECDH_Message/request',
            data: ...
        }
        """
        messageType = message['type']
        
        if messageType == 'CLIENT_RSA':
            return message['data']
        elif messageType == 'certificate':
            return message['data'] 
        elif messageType == 'ECDH_Parameters':
            # Maybe we should save these parameters for our own use...
            
            self.ELLIPTIC_CURVE = message['data'][0] 
            self.ELLIPTIC_CURVE_MODULUS = ...
            self.GENERATOR_POINT = ...
            self.GENERATOR_POINT_MODULUS = ...
            return message['data']
        elif messageType == 'ECDH_Message':
            if message['sender'] == 'client':
                # Might be useful to save this message as well..
                self.clientPublicPoint = ...
                return message['data']
            else:
                # Once we have the public message from the client and know one of the secrets, what can we do to 
                # recover the overall secret?
                # Remember to use self.fakeServerSecret as one of the ECDH secrets!
                
                fakeServerPoint = ...
                
                sharedSecretPoint = ...
                
                self.SHARED_SECRET_INT = pointToMessage(sharedSecretPoint)
                
                return (..., 0)
        elif messageType == 'request':
            if message['sender'] != 'client':
                pass
            
            firstRequestData = message['data']['data'] # Request data
            
            decrypted = json.loads(...) 
    
            self.CLIENT_PASSWORD = decrypted['password']
            
            return message['data']
        else:
            print('err', message)
            return ()

    def getPassword(self):
        return self.CLIENT_PASSWORD

Here's an example to test with:

In [ ]:
random.seed(0) # DON'T CHANGE THIS

# New client
client = Client()
server = Server()

# Initiate Interceptor
intercept = Interceptor()

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'CLIENT_RSA', 'data': client.RSA_PUBLIC_KEY})
if message:
    server.acceptConnection(message)
    
# Initiate Interceptor
intercept = Interceptor()

server_PK, signature = server.publishRSAPublicKey()

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'certificate', 'data': (server_PK, signature)})
    
assert client.verifyServerCertificate(*message)

# Once verified, ask for ECDH parameters
ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature = server.generateECDHParameters()

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'ECDH_Parameters', 'data': (ellipticCurve, ellipticCurveModulus,
                                                                                generatorPoint, generatorPointOrder, signature)})

assert client.verifyECDHParameters(*message)

# Once ECDH parameters are verified, generate our secret + public value
client_ECDH = client.generateSignedECDHMessage()
server_ECDH = server.generateSignedECDHMessage()

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'ECDH_Message', 'data': client_ECDH})
# server gets client ecdh
server.acceptECDHMessage(*message)

message = intercept.interceptMessage({'sender': 'server', 'receiver': 'client',
                                              'type': 'ECDH_Message', 'data': server_ECDH})
# client gets server ecdh 
client.acceptECDHMessage(*message)

# This is the encrypted signup request we see, generated by 
# client.generateRequest({'type': 'createLogin', 'username': 'admin', 'password': 'PASSWORD'})
# but for a different PASSWORD
signupRequest = {'data': '{"iv": "yC+cYmuOSA+1XVzdKm+f/w==", "ciphertext": "Y5s7r7JSJPdnTuQ57DFzepOvjz2DLDJB21eI4j5eX6fnPZdokcIiA8DTnwVon28Ia8ZlX+9odx3xVKTav+KCv9ullshBOH1yUHdU1Kir/ag="}',
 'hmac': 'b6d9d3a11ad7d1e16bda949c51432987ad42545fe82d1160621dbf78e1b3d0dd'}

message = intercept.interceptMessage({'sender': 'client', 'receiver': 'server',
                                              'type': 'request', 'data': signupRequest})

In [ ]:
# By now, we should have access to the client's password.
print('Recovered password:', intercept.getPassword())

### Submission

Once you've recovered the password, enter it into the Gradescope page [here](https://www.gradescope.com/courses/406561/assignments/2458295) and copy-paste your **Interceptor** class in the same assignment. (Unfortunately our usual autograder does not play nice with this style of challenge)